In [84]:
import parselmouth
import numpy as np
import pandas as pd

In [85]:
audio_file = "../data/raw_audio/AH_545880204-EE87D3E2-0D4C-4EAA-ACD7-C3F177AFF62F.wav"

try:
    # 1. Tải tệp âm thanh
    sound = parselmouth.Sound(audio_file)

    # 2. Tạo đối tượng PointProcess từ Sound
    # Lệnh "To PointProcess (periodic, cc)" cần các tham số f0_min và f0_max
    # Giá trị 75 và 500 là các tham số mặc định thường được dùng.
    point_process = parselmouth.praat.call(sound, "To PointProcess (periodic, cc)", 75, 500)

    # 3. Trích xuất Jitter từ đối tượng PointProcess
    jitter = parselmouth.praat.call(point_process, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)

    jitter_abs = parselmouth.praat.call(point_process, "Get jitter (local, absolute)", 0, 0, 0.0001, 0.02, 1.3)

    jitter_rap = parselmouth.praat.call(point_process, "Get jitter (rap)", 0, 0, 0.0001, 0.02, 1.3)

    jitter_ppq5 = parselmouth.praat.call(point_process, "Get jitter (ppq5)", 0, 0, 0.0001, 0.02, 1.3)

    jitter_ddp = parselmouth.praat.call(point_process, "Get jitter (ddp)", 0, 0, 0.0001, 0.02, 1.3)
    # 4. Trích xuất Shimmer từ cả Sound và PointProcess
    # Lệnh "Get shimmer (local)" yêu cầu cả hai đối tượng này.
    shimmer = parselmouth.praat.call([sound, point_process], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    dda_shimmer = parselmouth.praat.call([sound, point_process], "Get shimmer (dda)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq3Shimmer = parselmouth.praat.call([sound, point_process], "Get shimmer (apq3)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq5Shimmer = parselmouth.praat.call([sound, point_process], "Get shimmer (apq5)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq11Shimmer = parselmouth.praat.call([sound, point_process], "Get shimmer (apq11)", 0, 0, 0.0001, 0.02, 1.3, 1.6)


    print(f"Kết quả trích xuất cho tệp âm thanh: {audio_file}")
    print("-" * 50)
    print(f"Jitter cục bộ: {jitter:.4f} %")
    print(f"Shimmer cục bộ: {shimmer:.4f} %")
    print(f"Shimmer dda: {dda_shimmer:.4f} %")
    print(f"Shimmer apq3: {apq3Shimmer:.4f} %")
    print(f"Shimmer apq5: {apq5Shimmer:.4f} %")
    print(f"Shimmer apq11: {apq11Shimmer:.4f} %")
    print(f"Jitter cục bộ tuyệt đối: {jitter_abs:.8f} s")
    print(f"Jitter:RAP: {jitter_rap:.4f} %")
    print(f"Jitter:PPQ5: {jitter_ppq5:.4f} %")
    print(f"Jitter:DDP: {jitter_ddp:.4f} %")

except Exception as e:
    print(f"Đã xảy ra lỗi: {e}")

Kết quả trích xuất cho tệp âm thanh: ../data/raw_audio/AH_545880204-EE87D3E2-0D4C-4EAA-ACD7-C3F177AFF62F.wav
--------------------------------------------------
Jitter cục bộ: 0.0070 %
Shimmer cục bộ: 0.0876 %
Shimmer dda: 0.1479 %
Shimmer apq3: 0.0493 %
Shimmer apq5: 0.0559 %
Shimmer apq11: 0.0638 %
Jitter cục bộ tuyệt đối: 0.00006324 s
Jitter:RAP: 0.0036 %
Jitter:PPQ5: 0.0039 %
Jitter:DDP: 0.0109 %


In [86]:
import librosa
from scipy.stats import entropy

In [87]:
y, sr = librosa.load(audio_file, sr=None)

y = y / np.max(np.abs(y))

f0, voiced_flag, voice_probs = librosa.pyin(y=y, fmin=50, fmax=500, sr=sr)

f0 = f0[~np.isnan(f0)]

hist, _ = np.histogram(f0, bins=50, density=True)
ppe_raw = entropy(hist)              # entropy thô
ppe_norm = ppe_raw / np.log(len(hist))  # chuẩn hóa về [0, 1]
ppe_norm = entropy(hist, base=2) / np.log2(len(hist))

print(f'PPE: {ppe_norm}')

PPE: 0.47069250718855243


In [88]:
import numpy as np
from scipy.signal import medfilt

# ==== 1. Làm sạch pitch contour ====
def clean_f0(f0, kernel_size=5):
    # Bỏ NaN và giá trị <= 0
    f0 = f0[np.isfinite(f0)]
    f0 = f0[f0 > 0]
    if len(f0) < 10:
        raise ValueError("Pitch contour quá ngắn để tính DFA.")
    # Median filter để làm mượt
    f0_clean = medfilt(f0, kernel_size=kernel_size)
    return f0_clean

# ==== 2. Hàm tính DFA bậc 1 (cổ điển) ====
def dfa(signal):
    n_vals = np.floor(np.logspace(np.log10(4), np.log10(len(signal)//4), num=20)).astype(int)
    n_vals = np.unique(n_vals)
    flucts = []

    for n in n_vals:
        if n < 4:
            continue
        segments = len(signal) // n
        rms_vals = []
        for seg in range(segments):
            segment_data = signal[seg*n : (seg+1)*n]
            t = np.arange(n)
            coeffs = np.polyfit(t, segment_data, 1)
            trend = np.polyval(coeffs, t)
            detrended = segment_data - trend
            rms_vals.append(np.sqrt(np.mean(detrended**2)))
        flucts.append(np.mean(rms_vals))

    flucts = np.array(flucts)
    n_vals = np.array(n_vals[:len(flucts)])
    coeffs = np.polyfit(np.log(n_vals), np.log(flucts), 1)
    alpha = coeffs[0]
    return alpha

# ==== 3. Tính DFA với xử lý outlier ====
def compute_dfa_with_outlier_handling(f0, dfa_min=0.4, dfa_max=0.83, median_reference=0.65):
    try:
        f0_clean = clean_f0(f0)
        # Chuẩn hóa F0 (z-score)
        f0_norm = (f0_clean - np.mean(f0_clean)) / np.std(f0_clean)
        dfa_value = dfa(f0_norm)

        # ==== 4. Hậu xử lý DFA ====
        if dfa_value < dfa_min or dfa_value > dfa_max:
            print(f"[CẢNH BÁO] DFA {dfa_value:.3f} nằm ngoài range [{dfa_min}, {dfa_max}].")
            # Clip về range
            dfa_value = np.clip(dfa_value, dfa_min, dfa_max)
            # Hoặc thay bằng median nếu muốn ổn định hơn
            if dfa_value == dfa_min or dfa_value == dfa_max:
                dfa_value = median_reference
        return dfa_value
    except Exception as e:
        print("Lỗi khi tính DFA:", e)
        return np.nan


# ================== TEST ==================
# Ví dụ giả định f0 (pitch contour)
f0_example = np.random.normal(150, 20, 200)  # giả lập pitch
dfa_val = compute_dfa_with_outlier_handling(f0_example)
print("DFA (sau xử lý outlier):", dfa_val)


DFA (sau xử lý outlier): 0.4804682038685084


In [ ]:
import numpy as np
import librosa
from scipy.spatial.distance import pdist, squareform
import os
import warnings

# Bỏ qua các cảnh báo không cần thiết
warnings.filterwarnings('ignore', category=UserWarning, module='librosa')
warnings.filterwarnings('ignore', category=FutureWarning)

# --- HÀM TÍNH TOÁN RPDE (giữ nguyên) ---
def calculate_rpde(file_path, dimension=10, time_delay=1, threshold_ratio=0.2):
    """
    Hàm tính toán Recurrence Period Density Entropy (RPDE) từ một tệp âm thanh.
    """
    try:
        # 1. Tải và kiểm tra tệp âm thanh
        if not os.path.exists(file_path):
            print(f"Lỗi: Tệp không tồn tại tại đường dẫn '{file_path}'")
            return None

        y, sr = librosa.load(file_path, sr=None)
        
        if np.max(np.abs(y)) == 0 or len(y) < 2000:
            print("Lỗi: Tệp âm thanh im lặng hoặc quá ngắn để phân tích.")
            return None
        
        # 2. Chuẩn bị tín hiệu
        y = y / np.std(y)
        
        # 3. Tạo không gian pha
        if len(y) < (dimension - 1) * time_delay + 1:
            print("Lỗi: Tín hiệu quá ngắn để tạo không gian pha.")
            return None

        embedded_signal = np.array([y[i : i + (dimension * time_delay) : time_delay] 
                                    for i in range(len(y) - (dimension - 1) * time_delay)])

        # 4. Tính toán ma trận tái phát
        distance_matrix = squareform(pdist(embedded_signal, 'euclidean'))
        recurrence_matrix = (distance_matrix <= threshold_ratio).astype(int)
        
        # 5. Tính toán phân phối độ dài đường chéo
        diag_lengths = []
        for i in range(1, recurrence_matrix.shape[0]):
            diag = np.diag(recurrence_matrix, k=i)
            line_lengths = [len(s) for s in "".join(map(str, diag.astype(int))).split('0') if s]
            diag_lengths.extend(line_lengths)
        
        if not diag_lengths:
            return 0.0

        # 6. Tính toán RPDE
        hist = np.bincount(np.array(diag_lengths, dtype=int))
        hist = hist[hist > 0]
        probabilities = hist / np.sum(hist)
        rpde = -np.sum(probabilities * np.log2(probabilities))
        
        return rpde

    except Exception as e:
        print(f"Đã xảy ra lỗi không mong muốn: {e}")
        return None

# --- HÀM CHÍNH ĐỂ NHẬP LIỆU VÀ CHẠY PHÂN TÍCH ---
def main():
    """
    Hàm chính, hỏi người dùng đường dẫn file và in ra kết quả.
    """
    # Sử dụng một giá trị threshold_ratio mặc định hợp lý.
    # Dựa trên quá trình tinh chỉnh trước, 0.2 là một điểm khởi đầu tốt.
    default_threshold = 0.2
    
    print("--- CHƯƠNG TRÌNH PHÂN TÍCH RPDE TỪ TỆP ÂM THANH ---")
    print(f"(Sử dụng threshold_ratio mặc định là {default_threshold})")
    print("Nhập đường dẫn đến tệp WAV của bạn, hoặc nhấn Enter để thoát.")
    
    while True:
        # Hỏi người dùng nhập đường dẫn file
        audio_path = audio_file
        
        # Nếu người dùng không nhập gì và nhấn Enter, thoát chương trình
        if not audio_path:
            print("Đã thoát chương trình.")
            break
            
        # Gọi hàm tính toán
        rpde_value = calculate_rpde(audio_path, threshold_ratio=default_threshold)
        
        # In kết quả
        if rpde_value is not None:
            print("-" * 30)
            print(f"Tệp: {os.path.basename(audio_path)}")
            print(f"✅ Giá trị RPDE tính được là: {rpde_value:.4f}")
            print("-" * 30)

# Chạy hàm chính khi script được thực thi
if __name__ == '__main__':
    main()

--- CHƯƠNG TRÌNH PHÂN TÍCH RPDE TỪ TỆP ÂM THANH ---
(Sử dụng threshold_ratio mặc định là 0.2)
Nhập đường dẫn đến tệp WAV của bạn, hoặc nhấn Enter để thoát.


In [90]:
import parselmouth
from scipy.io import wavfile
from scipy.stats import entropy
import pywt

In [91]:
snd = parselmouth.Sound(audio_file)

point_process = parselmouth.praat.call(snd, "To PointProcess (periodic, cc)", 75, 500)

times = [parselmouth.praat.call(point_process, "Get time from index", i + 1)
         for i in range(parselmouth.praat.call(point_process, "Get number of points"))]

periods = np.diff(times)

periods_ms = np.array(periods)

mean_period_ms = np.mean(periods_ms)
std_period_ms = np.std(periods_ms)

print("Mean period (ms):", mean_period_ms)
print("Std dev period (ms):", std_period_ms)


Mean period (ms): 0.009152488197601135
Std dev period (ms): 0.0023783939643017217


In [92]:
def harmonicity_features(path, time_step=0.01, min_pitch=75, silence_threshold=0.03, periods_per_window=1.0):
    snd = parselmouth.Sound(path)
    harm = snd.to_harmonicity_ac(time_step=time_step,
                                 minimum_pitch=min_pitch,
                                 silence_threshold=silence_threshold,
                                 periods_per_window=periods_per_window)
    hnr_db = harm.values.ravel()
    valid = hnr_db > -200  # Praat dùng -200 dB cho khung vô thanh

    # Nếu không có khung hợp lệ, nới điều kiện và thử lại
    if not np.any(valid):
        harm = snd.to_harmonicity_ac(time_step=time_step,
                                     minimum_pitch=60,
                                     silence_threshold=0.0,
                                     periods_per_window=periods_per_window)
        hnr_db = harm.values.ravel()
        valid = hnr_db > -200
        if not np.any(valid):
            raise ValueError("Không có khung hữu thanh nào — hãy kiểm tra file hoặc giảm minimum_pitch/silence_threshold.")

    hnr_db_v = hnr_db[valid]

    # HNR (dB)
    mean_hnr = float(np.mean(hnr_db_v))

    # NHR (tỉ lệ)
    nhr = 10.0 ** (-hnr_db_v / 10.0)
    mean_nhr = float(np.mean(nhr))

    # r: “autocorrelation-like periodicity” trong (0,1)
    rpow = 10.0 ** (hnr_db_v / 10.0)
    r = rpow / (1.0 + rpow)
    mean_auto_corr = float(np.mean(r))
    r_min, r_max = float(np.min(r)), float(np.max(r))

    return mean_auto_corr, mean_hnr, mean_nhr, (r_min, r_max)

# Ví dụ dùng:
mean_auto_corr, mean_hnr, mean_nhr, (r_min, r_max) = harmonicity_features(audio_file)
print("meanAutoCorrHarmonicity (0..1):", mean_auto_corr)
print("meanHarmToNoiseHarmonicity (HNR dB):", mean_hnr)
print("meanNoiseToHarmHarmonicity (NHR):", mean_nhr)
print("periodicity r min/max:", r_min, r_max)

meanAutoCorrHarmonicity (0..1): 0.497483170064848
meanHarmToNoiseHarmonicity (HNR dB): 0.33372921888458257
meanNoiseToHarmHarmonicity (NHR): 1.3141374908859162
periodicity r min/max: 0.09066135803204405 0.9996425390519371
